<a href="https://colab.research.google.com/github/Srajan1122/Hyperspectral_Image_Classification/blob/srajan/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Importing Libraries

In [0]:
import pandas as pd
import numpy as np
from scipy import io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn import preprocessing
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten, Dropout, LSTM

In [0]:
loaded_dataset = io.loadmat('/content/drive/My Drive/Srajan/Internship/SEM4-LeadingindiaAI/Tasks/Datasets/Botswana/Botswana.mat')
for key, value in loaded_dataset.items():
  if isinstance(value, type(np.array([1]))):
    image = loaded_dataset[key]

In [0]:
ground_truth_1 = io.loadmat('/content/drive/My Drive/Srajan/Internship/SEM4-LeadingindiaAI/Tasks/Datasets/Botswana/Botswana_gt.mat')
for key, value in ground_truth_1.items():
  if isinstance(value, type(np.array([1]))):
    ground_truth = ground_truth_1[key]

## Resizing the data

In [0]:
image_with_gt = np.dstack((image, ground_truth))

In [0]:
final_output = image_with_gt.reshape(ground_truth.size, image.shape[2]+1)

## Data Visualization in pandas

In [0]:
data = pd.DataFrame(final_output)

In [8]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145
0,3996,3952,3698,3457,3063,2914,2796,2538,2629,2613,2533,2517,2306,2149,2065,1994,1943,1840,1779,1726,1654,1519,1542,1478,1404,1600,2755,2806,3459,4137,4008,2741,4144,4268,4069,4020,3537,3335,3582,3868,...,450,385,316,240,157,98,149,142,106,91,107,158,164,163,166,184,196,181,169,156,159,134,135,144,159,142,138,145,138,110,104,114,89,89,77,73,60,53,54,0
1,3996,3952,3698,3457,3063,2914,2796,2538,2629,2613,2533,2517,2306,2149,2065,1994,1943,1840,1779,1726,1654,1519,1542,1478,1404,1600,2029,2172,2672,3195,3184,2182,3316,3402,3199,3147,2763,2517,2841,3098,...,427,371,301,226,145,92,128,134,92,85,114,140,149,158,159,166,175,160,151,137,141,133,135,132,143,134,123,133,122,117,109,103,83,93,80,69,59,54,57,0
2,4100,3923,3615,3451,2941,2886,2850,2581,2628,2616,2607,2479,2359,2118,2099,1946,1971,1885,1743,1774,1680,1566,1534,1494,1375,1603,2152,2277,2870,3474,3502,2441,3612,3677,3489,3410,2991,2866,3101,3280,...,365,324,270,199,131,70,95,101,76,75,98,105,124,134,138,135,140,123,118,111,112,123,119,110,114,103,95,107,98,113,102,85,67,89,79,55,52,51,56,0
3,3908,3889,3594,3343,2975,2813,2665,2445,2532,2489,2423,2328,2209,2039,1944,1871,1850,1700,1616,1564,1493,1398,1356,1345,1271,1463,2046,2243,2939,3522,3582,2506,3674,3793,3590,3536,3125,2942,3194,3397,...,370,324,263,200,143,68,100,103,85,74,86,111,126,139,147,125,121,125,120,106,116,117,106,110,118,109,107,106,108,103,94,75,61,73,64,50,57,48,45,0
4,4007,3926,3767,3445,3038,2945,2914,2588,2616,2624,2608,2509,2347,2176,2099,2002,1982,1889,1775,1687,1705,1531,1530,1454,1450,1631,2222,2426,3174,3859,3855,2711,4073,4139,3992,3948,3447,3264,3517,3747,...,380,327,259,198,142,67,102,116,94,68,82,116,127,147,154,118,106,122,120,106,119,106,97,112,119,122,119,113,111,94,88,63,57,62,51,49,56,42,41,0


## Droping the rows if ground truth value is zero

In [9]:
print("Percentage of column which will be droped",(data.size - data[data.iloc[:, -1] == 0].size)/data.size,"%")

Percentage of column which will be droped 0.008595867208672087 %


In [0]:
zero_data = data.index[data.iloc[:, -1] == 0].tolist()
data = data[data.iloc[:, -1] != 0]

# Spliting the data into feature and target

In [0]:
X = data.iloc[:, :-1]
y = data.iloc[:,-1]

In [0]:
number_of_classes = len(np.unique(y))

# Feature Selection

In [0]:
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

# X = SelectKBest(f_classif, k=int((image.shape[2]+1)*0.75)).fit_transform(X, y)

# Feature Extraction 


In [0]:
from sklearn.decomposition import PCA 

pca = PCA(n_components = int((image.shape[2]+1)*0.80))
X = pca.fit_transform(X)

# OneHotEncoding in target column

In [0]:
# onehotencoder = OneHotEncoder() 
# y = onehotencoder.fit_transform(np.array(y).reshape(-1,1)).toarray() 

# Standardizing the feature columna

In [0]:
X = preprocessing.scale(X)

# Spliting the data into training and testing set 

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.60, random_state=1)#0.25 0.15

In [0]:
number = int(X_test.shape[0]/2)

xx_test = X_test[:number, :]
xx_val = X_test[number:, :]

if len(y_test.shape) > 1:
  loss = tensorflow.keras.losses.categorical_crossentropy
  metrics=['accuracy']
  yy_test = y_test[:number, :]
  yy_val = y_test[number:, :]
else:
  loss = tensorflow.keras.losses.sparse_categorical_crossentropy
  metrics=['sparse_categorical_accuracy']
  y_test = [x - 1 for x in y_test]
  y_train = [x - 1 for x in y_train]
  yy_test = y_test[:number]
  yy_val = y_test[number:]

In [0]:
from tensorflow.keras import backend as K
feature = X.shape[1]

# if K.image_data_format() == 'channels_first':
#     X_train = X_train.reshape(X_train.shape[0], 1, feature)
#     xx_test = xx_test.reshape(xx_test.shape[0], 1, feature)
#     xx_val = xx_val.reshape(xx_val.shape[0], 1, feature)
#     input_shape = (1, feature)
# else:
#     X_train = X_train.reshape(X_train.shape[0], feature, 1)
#     xx_test = xx_test.reshape(xx_test.shape[0], feature, 1)
#     xx_val = xx_val.reshape(xx_val.shape[0], feature, 1)
#     input_shape = (feature, 1)

X = X.reshape(X.shape[0], 1, feature)
X_train = X_train.reshape(X_train.shape[0], 1, feature)
xx_test = xx_test.reshape(xx_test.shape[0], 1, feature)
xx_val = xx_val.reshape(xx_val.shape[0], 1, feature)
input_shape = (1, feature)

# Genetic Algorithm

In [20]:
!pip install pyeasyga

  Created wheel for pyeasyga: filename=pyeasyga-0.3.1-py2.py3-none-any.whl size=6774 sha256=6db7ae1ba331f0a8616a0694967027af2722e2fa05626dcce5dd09f8ea231e8a
  Stored in directory: /root/.cache/pip/wheels/ef/cf/ef/7aff9fcd6c1e59dc276182f29a32e7c197665dd5eb547f30e6
Successfully built pyeasyga


In [0]:
# from pyeasyga import pyeasyga
# import random
# import numpy as np
# from sklearn.metrics import accuracy_score
# import keras
# import time

# count = 1

# def initilialize_population():
#     filter_1 = list(x for x in range(70, 120, 5))
#     filter_2 = list(x for x in range(70, 120, 5))

#     population = [filter_1, filter_2]

#     return list(population)

# def create_individual(data):
#   choice = []
#   for i in range(len(data)):
#     choice.append(random.choice(list(range(len(data[i])))))
#   return choice

# def fitness_function(individual, data):
#   global count
#   print('Individual no', count)
#   count += 1

#   model = Sequential()
#   model.add(LSTM(data[0][individual[0]], input_shape=input_shape, return_sequences=False))
#   model.add(Dropout(0.25))

#   model.add(Dense(number_of_classes, activation='softmax'))

#   model.compile(loss=loss,
#                 optimizer='adam',
#                 metrics=metrics)

#   model.fit(X_train, np.array(y_train),
#             batch_size=128,
#             epochs=20,
#             verbose=1,
#             validation_data=(xx_val,np.array(yy_val)))


#   score = model.evaluate(xx_test, np.array(yy_test), verbose=1)[1]
#   # prediction = model.predict(X_test)
#   # score = accuracy_score(y_test, prediction)
#   print('Score is', score)
#   return score
  
# data = initilialize_population()
# ga = pyeasyga.GeneticAlgorithm(seed_data=data,
#                                population_size=1,
#                                generations=1,
#                                crossover_probability=0.8,
#                                mutation_probability=0.4,
#                                elitism=True,
#                                maximise_fitness=True)
# ga.create_individual = create_individual
# ga.fitness_function = fitness_function

# start = time.time()
# ga.run()
# end = time.time()
# print('Time taken ------>', end-start)

In [0]:
# best_parameters = ga.best_individual()[1]
# for index,i in enumerate(data):
#   print(i[best_parameters[index]])

# print(ga.best_individual()[0])

# Building RNN Model

In [0]:
# # design network
# model = Sequential()
# model.add(LSTM(data[0][best_parameters[0]], input_shape=input_shape, return_sequences=False))
# model.add(Dropout(0.25))

# model.add(Dense(number_of_classes, activation='softmax'))
# model.compile(loss=loss, optimizer='adam', metrics=metrics)
# model.summary()

# # fit network
# history = model.fit(X_train, np.array(y_train), validation_data=(xx_val, np.array(yy_val)), 
#                     epochs=25,verbose=1)

In [24]:
model = Sequential()
model.add(LSTM(150, input_shape=input_shape, return_sequences=False))
model.add(Dropout(0.25))

model.add(Dense(number_of_classes, activation='softmax'))
model.compile(loss=loss, optimizer='adam', metrics=metrics)
model.summary()

# fit network
history = model.fit(X_train, np.array(y_train), validation_data=(xx_val, np.array(yy_val)), 
                    epochs=25,verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 150)               160200    
_________________________________________________________________
dropout (Dropout)            (None, 150)               0         
_________________________________________________________________
dense (Dense)                (None, 14)                2114      
Total params: 162,314
Trainable params: 162,314
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
41/41 [==============================] - 1s 20ms/step - loss: 2.5281 - sparse_categorical_accuracy: 0.2140 - val_loss: 2.3794 - val_sparse_categorical_accuracy: 0.5128
Epoch 2/25
41/41 [==============================] - 0s 7ms/step - loss: 2.1372 - sparse_categorical_accuracy: 0.7090 - val_loss: 2.0579 - val_sparse_categorical_accuracy: 0.7569
Epoch 3/25
41/41 [=====

In [25]:
score = model.evaluate(xx_test, np.array(yy_test), verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

31/31 [==============================] - 0s 3ms/step - loss: 0.2248 - sparse_categorical_accuracy: 0.9374
Test loss: 0.22478052973747253
Test accuracy: 0.9373716711997986


In [0]:
if len(np.array(y_test).shape) == 1:
  y_test = [x+1 for x in y_test]
  y_train = [x+1 for x in y_train]

In [0]:
# pred = X.reshape((X.shape[0], 1, X.shape[1]))



# Y_pred_test = model.predict(pred)
# y_pred_test = np.argmax(Y_pred_test, axis=1)

# from sklearn.metrics import accuracy_score
# from sklearn.metrics import classification_report
# import matplotlib.pyplot as plt
# print(accuracy_score(y, y_pred_test)*100)

# #= summarize history for accuracy
# plt.plot(history.history['sparse_categorical_accuracy'])
# plt.plot(history.history['val_sparse_categorical_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [28]:
!pip install spectral

     |████████████████████████████████| 194kB 2.8MB/s 


In [0]:
import spectral as sp
prediction = model.predict(X)

if not len(prediction.shape) == 2:
    y_test = [x + 1 for x in y_test]
    y_train = [x + 1 for x in y_train]

if len(prediction.shape) == 2:
    predicted_gt_1 = np.argmax(prediction, axis=1)
    predicted_gt_1_list = list(predicted_gt_1)
    predicted_gt_1_list = [x + 1 for x in predicted_gt_1_list]
else:
    predicted_gt_1_list = prediction

for i in zero_data:
    predicted_gt_1_list.insert(i, 0)

predicted_gt_1_list = np.array(predicted_gt_1_list).reshape(image.shape[0], image.shape[1])
sp.save_rgb('predicted_gt.jpg', predicted_gt_1_list, colors=sp.spy_colors)
